In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L3\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['AT.csv', 'BE.csv', 'BG.csv', 'CH.csv', 'CZ.csv', 'DK.csv', 'ES.csv', 'FR.csv', 'GR.csv', 'IT.csv', 'NL.csv', 'PT.csv', 'SI.csv', 'SK.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[5]))

,Demand,Normalized_net,Temperature,DNI,DHI
0,0.00,0.000000,3.983,0.0,0.0
1,3100.02,0.360299,4.273,0.0,0.0
2,2980.39,0.346395,4.563,0.0,0.0
3,2933.49,0.340944,4.674,0.0,0.0
4,2941.53,0.341878,4.635,0.0,0.0
...,...,...,...,...,...
43819,3785.27,0.439942,1.961,0.0,0.0
43820,3582.21,0.416341,1.874,0.0,0.0
43821,3438.26,0.399610,1.957,0.0,0.0
43822,3309.25,0.384616,2.213,0.0,0.0


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[5]

zoneData = pd.read_csv(os.path.join(path, fileName), usecols= ["Demand", "Temperature"])
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
# zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_DK,Temp_DK
0,0.00,3.983
1,3100.02,4.273
2,2980.39,4.563
3,2933.49,4.674
4,2941.53,4.635
...,...,...
43819,3785.27,1.961
43820,3582.21,1.874
43821,3438.26,1.957
43822,3309.25,2.213


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_DK,Temp_DK,Lag-2,Lag-7
0,3300.30,4.104,3137.68,0.00
1,3277.88,3.889,3102.22,3100.02
2,3300.41,3.640,3088.44,2980.39
3,3375.16,3.446,3168.02,2933.49
4,3634.32,3.534,3426.03,2941.53
...,...,...,...,...
43651,3785.27,1.961,4158.58,3510.73
43652,3582.21,1.874,4001.25,3358.03
43653,3438.26,1.957,3786.00,3243.88
43654,3309.25,2.213,3558.11,3194.35


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 23s 16ms/step - loss: 0.0067 - mape: 3950.6436 - mae: 0.0643 - val_loss: 0.0045 - val_mape: 20.9946 - val_mae: 0.0507
Epoch 2/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0029 - mape: 2374.8716 - mae: 0.0406 - val_loss: 0.0032 - val_mape: 20.4883 - val_mae: 0.0453
Epoch 3/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0024 - mape: 2416.5383 - mae: 0.0364 - val_loss: 0.0085 - val_mape: 32.2107 - val_mae: 0.0784
Epoch 4/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0021 - mape: 2132.9392 - mae: 0.0340 - val_loss: 0.0069 - val_mape: 30.5897 - val_mae: 0.0723
Epoch 5/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0020 - mape: 1867.2500 - mae: 0.0329 - val_loss: 0.0074 - val_mape: 32.3343 - val_mae: 0.0742
Epoch 6/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0020 - mape: 1545.7258 - mae: 0.0321 - val_l

1105/1105 [==============================] - 17s 15ms/step - loss: 9.8731e-04 - mape: 1039.1841 - mae: 0.0226 - val_loss: 0.0066 - val_mape: 29.1028 - val_mae: 0.0689
Epoch 48/100
1105/1105 [==============================] - 17s 15ms/step - loss: 9.8677e-04 - mape: 1248.9750 - mae: 0.0226 - val_loss: 0.0052 - val_mape: 25.6448 - val_mae: 0.0586
Epoch 49/100
1105/1105 [==============================] - 17s 15ms/step - loss: 9.6745e-04 - mape: 1003.0936 - mae: 0.0224 - val_loss: 0.0050 - val_mape: 27.3410 - val_mae: 0.0587
Epoch 50/100
1105/1105 [==============================] - 17s 15ms/step - loss: 9.5632e-04 - mape: 960.1555 - mae: 0.0223 - val_loss: 0.0056 - val_mape: 26.4664 - val_mae: 0.0606
Epoch 51/100
1105/1105 [==============================] - 17s 15ms/step - loss: 9.3991e-04 - mape: 1156.6519 - mae: 0.0220 - val_loss: 0.0046 - val_mape: 24.9841 - val_mae: 0.0559
Epoch 52/100
1105/1105 [==============================] - 17s 15ms/step - loss: 9.5002e-04 - mape: 1136.1167 - mae

Epoch 93/100
1105/1105 [==============================] - 17s 16ms/step - loss: 6.1276e-04 - mape: 732.4505 - mae: 0.0181 - val_loss: 0.0048 - val_mape: 23.7693 - val_mae: 0.0540
Epoch 94/100
1105/1105 [==============================] - 17s 16ms/step - loss: 6.1266e-04 - mape: 955.5308 - mae: 0.0181 - val_loss: 0.0055 - val_mape: 25.3720 - val_mae: 0.0601
Epoch 95/100
1105/1105 [==============================] - 17s 16ms/step - loss: 6.1598e-04 - mape: 1020.9557 - mae: 0.0181 - val_loss: 0.0054 - val_mape: 26.2775 - val_mae: 0.0605
Epoch 96/100
1105/1105 [==============================] - 17s 16ms/step - loss: 6.0626e-04 - mape: 829.7574 - mae: 0.0180 - val_loss: 0.0063 - val_mape: 28.4146 - val_mae: 0.0650
Epoch 97/100
1105/1105 [==============================] - 17s 15ms/step - loss: 6.0185e-04 - mape: 883.6456 - mae: 0.0179 - val_loss: 0.0065 - val_mape: 28.9613 - val_mae: 0.0660
Epoch 98/100
1105/1105 [==============================] - 18s 16ms/step - loss: 5.8658e-04 - mape: 803.3

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 1854.821475982666


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 11ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.09464108, 0.10197148, 0.12213306, ..., 0.15921584, 0.12174509,
       0.09730389], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.13871727, 0.13797705, 0.12397894, ..., 0.2066807 , 0.18686834,
       0.18789113])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,2995.71,2708.703857
1,2990.89,2756.436279
2,2899.74,2887.720459
3,2988.63,3036.507324
4,3380.95,3374.795898
...,...,...
4363,3785.27,3536.033936
4364,3582.21,3355.729492
4365,3438.26,3129.188232
4366,3309.25,2885.194092


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

6.385112860849886